In [2]:
import pandas as pd
from helpermodules import memory_handling as mh
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from functions import compute_sentiment, filtering_df, retrieve_datas, scraping_speeches, update_realtime, analysis
from functions.update_realtime import change_time
from functions.filtering_df import main as filtering
from functions.analysis import main as plot
from functions.scraping_speeches import main as scraping

#######
'''questa prima parte del codice presenta il codice che dovrebbe essere implementato 
(quello preceduto da '#') e in seguito il codice che ho utlizzato io non avendo ancora 
accesso alle funzioni, riciclando i file csv e 'pulendoli' per la task.
Le funzioni che verranno implementate devono restituire un oggetto che abbia 
la stessa forma e caratteristiche dell'oggetto finale (evidenziato nel codice)'''

yearlist = [2020,2021,2022,2023,2024]
# df_fed = scraping(yearlist) -> ALREADY RUN, STORE IN ANOTHER FILE
file = "2020-2024fedspeeches.pkl"
helper = mh.PickleHelper.pickle_load(file)
df_fed = helper.obj

timezone1 = df_fed

#df_prices = retrieve_datas(df_speech, deltabefore, deltaafter) -> ALREADY RUN
file2 = "2020-2024prices.pkl"
helper = mh.PickleHelper.pickle_load(file2)
df = helper.obj
timezone2 = df
print(df.columns) 
df['date'] = df['datetime'].dt.date
df_prices = df
'''df['datetime'] = df.apply(
    lambda row: row['datetime'].tz_localize('UTC').tz_convert(row['timezone']), axis=1
)
# Drop the 'timezone' column as it's no longer needed
df = df.drop(columns=['timezone'])
df['date'] = df['datetime'].dt.date
df['datetime'] = df['datetime'] + pd.Timedelta(hours=4) # updated to the correct value
df_prices = df #<--- OGGETTO FINALE
timezone22 = df_prices'''


#df_speech = computespeech()
file3 = "2020-2024speeches.pkl"
helper = mh.PickleHelper.pickle_load(file3)
df_speech = helper.obj

df_speech = df_speech[df_speech['date']>='2020-01-01']
df_speech = df_speech.sort_values(['date','timestamp'], ascending=True) #<--- OGGETTO FINALE
timezone3 = df_speech


#df_sentiment = compute_sentiment()
file4 = "2020-2024sentiment.pkl"
helper = mh.PickleHelper.pickle_load(file4)
df_sentiment = helper.obj 


############

df_fed.rename(columns={'timestamp': 'opening_time'}, inplace=True)
#update the correct timestamp for df_speech
df_speech_final = change_time(df_speech, df_fed)
#sorting the values
df_speech_final = df_speech_final.sort_values(['date','timestamp'], ascending=True)



# Define market open and close times (in ET)
market_open = pd.to_datetime('09:30:00', format='%H:%M:%S').time()
market_close = pd.to_datetime('16:00:00', format='%H:%M:%S').time()

# Extract time from 'timestamp'
df_speech_final['time'] = df_speech_final['timestamp'].dt.time

# Filter rows where 'time' is within market hours
df_filtered = df_speech_final[(df_speech_final['time'] >= market_open) & (df_speech_final['time'] < market_close)]
df_speech_final = df_filtered.drop(columns=['time'])



df_speech=df_speech_final



Index(['datetime', 'close', 'volume'], dtype='object')
The drop ratio is 78.27298050139275 %


In [4]:
def calculate_speech_durations(df_speech):
    """
    Calculates the duration of each speech by counting the number of rows for each unique
    combination of 'date', 'speaker', and 'speech'.

    Parameters:
    df_speech : pandas.DataFrame
        A dataframe containing ['date', 'timestamp', 'speaker', 'speech'] columns.

    Returns:
    pandas.DataFrame
        A dataframe with a new 'duration' column added, indicating the length of each speech.
    """
    # Ensure the 'date' column is in datetime format
    df_speech['date'] = pd.to_datetime(df_speech['date'])

    # Group by 'date', 'speaker', and 'speech' and count rows to calculate durations
    speech_durations = df_speech.groupby(['date', 'speaker', 'title']).size().reset_index(name='duration')
    speech_durations = speech_durations[speech_durations['duration']>=5]
    # Merge the calculated durations back into the original DataFrame
    df_speech = df_speech.merge(speech_durations, on=['date', 'speaker', 'title'], how='right')


    return df_speech

def find_timestart(df_speech):
    """
    Identifies the earliest timestamp for each speech.

    Parameters:
    df_speech : pandas.DataFrame
        A dataframe containing ['date', 'timestamp', 'speaker', 'speech'] columns.

    Returns:
    pandas.DataFrame
        A dataframe containing only the first timestamp for each unique speech.
    """
    df_speech['timestamp'] = pd.to_datetime(df_speech['timestamp'])

    # Get the row with the minimum 'timestamp' for each group of 'date', 'speech', and 'speaker'
    grouped_df = (
        df_speech.loc[df_speech.groupby(['date', 'title', 'speaker'])['timestamp'].idxmin()]
        .reset_index(drop=True)
    )

    return grouped_df


In [5]:
def filtering_function(df_prices, df_speech, deltabefore=0, deltaafter=0):
    """
    Filters df_prices by retaining only rows where 'timestamp' falls within the time range
    of speeches in df_speech, including optional buffers before and after the speech duration.

    Parameters:
    df_prices : pandas.DataFrame
        A dataframe containing ['date', 'datetime', 'close', 'volume'] columns.
    df_speech : pandas.DataFrame
        A dataframe containing ['date', 'timestamp', 'speaker', 'speech', 'duration'] columns.
    deltabefore : int, optional
        Time in minutes to include before the start of each speech (default is 0).
    deltaafter : int, optional
        Time in minutes to include after the end of each speech (default is 0).

    Returns:
    pandas.DataFrame
        A filtered dataframe containing rows from df_prices within the speech time ranges.
    """
    # Ensure datetime columns are in the correct format
    df_prices['datetime'] = pd.to_datetime(df_prices['datetime'])

    # Prepare durations and select earliest timestamps
    durations_df = calculate_speech_durations(df_speech)
    durations_df = find_timestart(durations_df)
    durations_df['timestamp'] = durations_df.apply(
        lambda row: row['timestamp'].replace(year=row['date'].year, month=row['date'].month, day=row['date'].day),
        axis=1
    )

    # Initialize list for storing filtered rows
    filtered_rows = []

    # Iterate over each speech and filter df_prices accordingly
    for _, speech in durations_df.iterrows():
        start_time = pd.to_datetime(speech['timestamp']) - pd.Timedelta(minutes=deltabefore)
        duration = speech['duration']
        end_time = pd.to_datetime(speech['timestamp']) + pd.Timedelta(minutes=duration + deltaafter)
        print(start_time,end_time,'\n\n')
        # Filter rows based on time range
        mask = (df_prices['datetime'] >= start_time) & (df_prices['datetime'] <= end_time)
        filtered_subset = df_prices[mask].copy()

        # Add speech-related details
        filtered_subset['title'] = speech['title']
        filtered_subset['speaker'] = speech['speaker']
        filtered_rows.append(filtered_subset)

    # Combine filtered rows into a single dataframe     
    filtered_df = pd.concat(filtered_rows, ignore_index=True)

    return filtered_df



In [ ]:
deltabefore = 0
deltaafter = 0 
df_speech_final = pd.merge(
    df_speech, 
    df_sentiment[['text_by_minute', 'finbert_score', 'speaker']], 
    on=['text_by_minute', 'speaker'], 
    how='left'
)

# eliminate duplicates 
df_speech_final = df_speech_final.drop_duplicates(subset='timestamp', keep='first')

df_speech_final.rename(columns={'speech': 'title'}, inplace=True)


# Filter prices based on speech data
df_prices_final = filtering_function(df_prices, df_speech_final, deltabefore, deltaafter)


# Calculate percentage change in price and merge with speech data
df_prices_final.rename(columns={'datetime': 'timestamp'}, inplace=True)
df_prices_final['timestamp'] = pd.to_datetime(df_prices_final['timestamp'])
def calc_pct_change(group):
    # Sort the group by timestamp
    group = group.sort_values('timestamp')
    
    # Calculate pct_change on 'close'
    group['pct_change'] = group['close'].pct_change()
    
    return group

# Apply the custom function on each group defined by 'title' and 'date'
df_prices_final = df_prices_final.groupby(['title', 'date'], group_keys=False).apply(calc_pct_change)
df_speech_final.rename(columns={'speech':'title'}, inplace=True)


df_speech_final['timestamp'] = df_speech_final.apply(
    lambda row: row['timestamp'].replace(year=row['date'].year, month=row['date'].month, day=row['date'].day),
    axis=1
)
df_speech_final['date'] = pd.to_datetime(df_speech_final['date'])
df_prices_final['date'] = pd.to_datetime(df_prices_final['date'])


#make sure they are both sorted by date

df_speech_final = pd.merge(
    df_speech_final,
    df_prices_final[['date', 'title', 'timestamp', 'pct_change','volume','close']],
    on=['date', 'title', 'timestamp'],
    how='right'
)






2020-02-21 13:30:00-05:00 2020-02-21 13:44:00-05:00 


2020-02-25 15:00:00-05:00 2020-02-25 15:06:00-05:00 


2020-05-21 12:00:00-04:00 2020-05-21 12:14:00-04:00 


2020-11-17 13:00:00-05:00 2020-11-17 13:15:00-05:00 


2021-02-22 15:30:00-05:00 2021-02-22 15:37:00-05:00 


2021-02-24 13:00:00-05:00 2021-02-24 13:10:00-05:00 


2021-03-18 10:55:00-04:00 2021-03-18 11:01:00-04:00 


2021-04-14 14:45:00-04:00 2021-04-14 15:02:00-04:00 


2021-05-03 13:20:00-04:00 2021-05-03 13:29:00-04:00 


2021-08-03 13:00:00-04:00 2021-08-03 13:07:00-04:00 


2021-09-09 12:00:00-04:00 2021-09-09 12:16:00-04:00 


2021-09-28 12:40:00-04:00 2021-09-28 13:01:00-04:00 


2021-10-12 10:15:00-04:00 2021-10-12 10:24:00-04:00 


2021-11-19 10:45:00-05:00 2021-11-19 11:07:00-05:00 


2021-11-19 12:15:00-05:00 2021-11-19 12:41:00-05:00 


2021-11-30 13:00:00-05:00 2021-11-30 13:22:00-05:00 


2022-09-07 13:00:00-04:00 2022-09-07 13:21:00-04:00 


2022-09-28 10:00:00-04:00 2022-09-28 10:29:00-04:00 


2022-10-06

/var/folders/l4/ybqxqy391fx0g8rhn62_7n1h0000gn/T/ipykernel_1875/3120347034.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_prices_final = df_prices_final.groupby(['title', 'date'], group_keys=False).apply(calc_pct_change)
